# Agent Deployment on AgentCore Runtime

## 1. Environment Setup

### 1.1 Load Configuration Variables

In [ ]:
from config import (
    SECRETS_NAME, PARAMETER_NAME
)

print(f"✅ SECRET_ID: {SECRETS_NAME}")
print(f"✅ SSM_AGENT_ARN_PARAM: {PARAMETER_NAME}")

### 1.2 Check Required Files

In [ ]:
import os

required_files = [
    'support_case_agent.py',
    'requirements.txt',
]

try:
    for file in required_files:
        print(f"Checking: {file}")

        if not os.path.exists(file):
            raise FileNotFoundError(f"Required file {file} not found")

    print("✅ All required files found")
except FileNotFoundError as e:
    print(f"❌ {e}")
    raise

## 2. AgentCore Runtime Configuration

### 2.1 Initialize AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

boto_session = Session()
region = boto_session.region_name
print(f"✅ Using AWS region: {region}")

agentcore_runtime = Runtime()
print("✅ AgentCore Runtime initialized")

### 2.2 Create IAM Execution Role

In [ ]:
from aws_setup import get_or_create_role
from config import AGENT_NAME

print("Getting or creating IAM role...")
execution_role_arn = get_or_create_role(AGENT_NAME)
print(f"✅ Using role: {execution_role_arn}")

### 2.3 Configure Agent

## 修改support_case_agent.py，同样修改SECRET_ID和SSM_AGENT_ARN_PARAM变量与MCP中的SECRETS_NAME和SECRETS_NAME保持一致

In [ ]:
from config import AGENT_NAME

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="support_case_agent.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=AGENT_NAME
)
print("✅ Configuration completed")
print(f"Response: {response}")

## 3. Agent Deployment

### 3.1 Start Runtime

In [ ]:
msg = "Launching Agent to AgentCore Runtime (This might take several minutes...)"
print(msg)
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)
print("✅ Launch completed")

print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### 3.2 Check Runtime Status

In [ ]:
import time

print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
max_wait_time = 1800  # 30 minutes
wait_time = 0

while status not in end_status and wait_time < max_wait_time:
    msg = f"Status: {status} - waiting... ({wait_time}s/{max_wait_time}s)"
    print(msg)
    time.sleep(10)
    wait_time += 10

    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if wait_time >= max_wait_time:
    print(f"⚠️ Timeout reached. Current status: {status}")
elif status == 'READY':
    print(f"✅ AgentCore Runtime is READY!")
else:
    print(f"⚠️ AgentCore Runtime status: {status}")

print(f"Final status: {status}")

## ✅ Deployment Complete

The Agent has been successfully deployed on AgentCore Runtime with all required permissions.

## 4. Test & Validate

### 4.1 Test Support Case Agent

In [ ]:
from support_case_agent_test import test_support_case_agent

# Test the deployed support case agent\n
test_support_case_agent()